In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/mobile-price-classification/train.csv
/kaggle/input/mobile-price-classification/test.csv


Mobile Price Classification

In [2]:
# Création de mes datasets. 

#Train pour le entrainer mon algo 
df_train = pd.read_csv("../input/mobile-price-classification/train.csv")

# Vérification des caracteristique du dataset
df_train.head()

,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,...,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi,price_range
0,842,0,2.2,0,1,0,7,0.6,188,2,...,20,756,2549,9,7,19,0,0,1,1
1,1021,1,0.5,1,0,1,53,0.7,136,3,...,905,1988,2631,17,3,7,1,1,0,2
2,563,1,0.5,1,2,1,41,0.9,145,5,...,1263,1716,2603,11,2,9,1,1,0,2
3,615,1,2.5,0,0,0,10,0.8,131,6,...,1216,1786,2769,16,8,11,1,0,0,2
4,1821,1,1.2,0,13,1,44,0.6,141,2,...,1208,1212,1411,8,2,15,1,1,0,1


In [3]:
# Test pour tester mon algo
df_test = pd.read_csv("../input/mobile-price-classification/test.csv")

# Vérification des caracteristique du dataset
df_test.head()

,id,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,...,pc,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi
0,1,1043,1,1.8,1,14,0,5,0.1,193,...,16,226,1412,3476,12,7,2,0,1,0
1,2,841,1,0.5,1,4,1,61,0.8,191,...,12,746,857,3895,6,0,7,1,0,0
2,3,1807,1,2.8,0,1,0,27,0.9,186,...,4,1270,1366,2396,17,10,10,0,1,1
3,4,1546,0,0.5,1,18,1,25,0.5,96,...,20,295,1752,3893,10,0,7,1,1,0
4,5,1434,0,1.4,0,11,1,49,0.5,108,...,18,749,810,1773,15,8,7,1,0,1


In [4]:
# Check des colonnes pour selectionnner mes features
df_test.columns
 #df_train.columns

Index(['id', 'battery_power', 'blue', 'clock_speed', 'dual_sim', 'fc',
       'four_g', 'int_memory', 'm_dep', 'mobile_wt', 'n_cores', 'pc',
       'px_height', 'px_width', 'ram', 'sc_h', 'sc_w', 'talk_time', 'three_g',
       'touch_screen', 'wifi'],
      dtype='object')

In [5]:
# Création de ma target en 'y' qui est le prix
y = df_train["price_range"]

# Récupération des features que j'ai supposées être bonnes pour la prédiction de la target
features = ['battery_power','int_memory','pc','ram','touch_screen','wifi']

# Création de ma variable X qui contiendra toutes les features précédentes depuis le dataset 'df_train'
X = df_train[features]


Le bloc si dessous c'est pour tester sur les valeurs d'entrainement en randomforest une prédiction et le coût erreur

In [6]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split

# je vais split le df train pour m'entrainer (une partie pour entrainer l'algo et l'autre pour tester dessus)
train_X, val_X, train_y, val_y = train_test_split(X, y, random_state = 1)

# Création de ma variable random forest
train_model = RandomForestRegressor(random_state = 1)

# Je fit mes features et la target de test avec ma variable random forest
train_model.fit(train_X, train_y)

# et je tente de prédire sur les features 'réelles'
main_prediction = train_model.predict(val_X)

# verification du coût erreur entre ma prédiction et ma 'réelle' target
main_mae = mean_absolute_error(main_prediction, val_y)
print("Mon taux d'erreur en random forest est de :", main_mae)

Mon taux d'erreur en random forest est de : 0.2672


Le bloc ci dessous sera pour tester ma prediction sur des valeurs 'réelles'

In [7]:
# Création de mon dataset real_X qui contiendra toutes les features précédentes depuis le dataset 'df_test'
real_X = df_test[features]

# Création de ma nouvelle variable random forest
real_model = RandomForestRegressor(random_state = 1)

# Fit du dataset 'train' complet et de la target
real_model.fit(X, y)

# Prédiction sur mes nouvelles features 'real_X'
real_prediction = real_model.predict(real_X)
print(real_prediction)

[2.98 2.99 2.1  3.   1.03 3.   3.   0.3  2.97 0.08 3.   3.   0.   0.02
 1.94 0.2  1.76 1.48 2.98 2.11 0.68 2.52 1.17 1.28 3.   0.   2.2  0.
 2.41 0.05 2.33 0.07 2.88 0.01 0.   1.11 2.7  1.5  2.64 1.43 0.58 1.96
 0.01 0.02 0.09 0.77 0.91 2.25 1.02 2.62 1.45 0.   2.67 0.42 3.   0.98
 2.86 1.17 0.93 3.   3.   2.83 0.12 2.   0.99 0.89 1.   2.28 0.97 1.42
 1.09 1.82 2.18 3.   3.   0.   2.02 0.13 1.93 3.   0.84 3.   3.   0.01
 3.   0.   2.99 1.03 2.94 0.22 1.04 1.63 2.48 0.03 2.02 1.72 0.04 1.94
 1.04 2.62 0.98 0.   0.   2.57 0.94 2.5  0.   1.1  2.04 3.   3.   2.48
 1.31 3.   2.44 3.   3.   1.26 2.99 0.   0.02 3.   1.49 1.5  1.11 0.03
 3.   2.43 2.52 1.06 0.   1.57 1.49 1.08 2.71 0.25 1.99 0.   2.99 2.05
 1.05 2.96 1.35 2.03 2.61 2.87 2.97 2.58 1.25 3.   2.12 3.   0.28 0.17
 2.55 1.9  3.   3.   3.   3.   2.34 2.07 3.   2.5  2.7  3.   0.64 0.
 3.   0.   0.   0.27 1.21 0.3  0.   1.04 0.85 0.   0.88 2.42 0.31 0.
 0.1  0.   1.46 2.06 1.52 0.17 0.02 0.32 0.   0.26 0.53 2.96 1.48 0.47
 2.11 1.93 2

In [8]:
output = pd.DataFrame({'Id': df_test.id,
                       'SalePrice': real_prediction})
output.to_csv('submission.csv', index=False)